# Plot and Analyze 

---

## Overview

In this document we'll take our premade cloud regimes and create plots of the cluster centers and spatial distributions. You'll also learn how to extract data about a specific cloud regime. Yes, we've already made plenty of plots of cluster centers and spatial distributions, however this script does it differently. This script preforms NO CLUSTERING. It only takes a set of premade cloud regimes and then fits the data it is passed into them. This is MUCH faster than preforming the clustering again and allows one to apply cloud regimes to sperate data. For example, we created CRs from ISCCP observation data in this tutorial. Using this script, you could apply those CRs to data produced by models running with ISCCP satellite simulator and examine them. You’d do this by setting data_path as a path to the model data, but still passing the ISCCP observation cloud regimes to the premade_cloud_regimes variable. Also, at the end of the script you'll learn how to use the outputs to further explore any other variables of interest in relation to your cloud regimes. 

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intermediate NumPy](https://foundations.projectpythia.org/core/numpy/intermediate-numpy.html) | Necessary | |
| [Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |
| [Introduction to Pandas](https://foundations.projectpythia.org/core/pandas/pandas.html) | Helpful | Familiarity with labeled data |

- **Time to learn**: 15 minutes


---

## Imports

In [1]:
from Functions import plot_hists, plot_rfo, open_and_process
import logging as lgr
import xarray as xr
import numpy as np

## Defining Variables

Here we define the variables necessary to begin using the toy ISCCP dataset included with this cookbook. To start off leave these variables alone, but later on feel free to experiment. 

If running locally with your own dataset, you will need to change many of variables these to match and point towards your data.

In [ ]:
# Path to data to cluster
data_path = "./ISCCP_toy_data.nc"

# Path to the directory you wish to save plots in if running as a script, if None plots will only be shown and not saved. Enter as String
save_path = None
save_prefix = None  # prefix to put into the name of each plot to distinguish different runs of this script

# Variable name of data to cluster in data_path
# Name of tau dimension for var_name
# Name of height/pressure dimension for var_name
var_name =  'n_pctaudist' 
tau_var_name =  'levtau' 
ht_var_name =  'levpc'
lat_var_name = 'lat'
lon_var_name = 'lon'

# Does this dataset use cloud top height or cloud top pressure? enter "h" for height or "p" for pressure
height_or_pressure = 'p'

# The premade cloud regimes to use. The script will then fit the histograms located in the files in data_path into these CRs.
# Set this variable to a path to a numpy ndarray, or an xarray data array of premade cloud regimes of shape=(k, n_tau_bins * n_pressure_bins)
premade_cloud_regimes = "./toy_ISCCP_cluster_centers.npy"

# Choose whether to use a euclidean or wasserstein distance
wasserstein_or_euclidean = "euclidean"

# Minimum and Maximum longitudes and latitudes entered as list, or None for entire range
lat_range = [-90,90]
lon_range = [-180,180]

# Time Range min and max, or None for all time, entered as list of str: Ex. ["2003-03-01", "2004-07-01"] or ['2003','2007']
time_range = None

# Use data only over land or over ocean
# Set to 'L' for land only, 'O' for ocean only, or False for both land and ocean
only_ocean_or_land = False
# Does this dataset have a built in variable for land fraction? if so enter as a string, otherwise cartopy will be used to mask out land or water
land_frac_var_name = None

# Logging level, set to "INFO" for information about what the code is doing, otherwise keep at "WARNING"
logging_level = 'INFO'

# Setting up logger
lgr.root.setLevel(logging_level)
# Concatenating save_path and save prefix
if save_path != None: save_path = save_path + save_prefix

## Opening, Labeling and Plotting
Here we open our data and assign each histogram to the nearest cluster. We store this information in an array named cluster_labels. After this we call plot_hists() to create plots of the CR centers, then we call plot_rfo() to create plots of the spatial distribution of each CR.

In [ ]:
# Automatically setting k from the premade cloud regimes
k = len(np.load(premade_cloud_regimes))
# Setting k-means properteis to None since they arent used but still must be passed
tol, max_iter, init, n_init, gpu = None, None, None, None, None
# Opening data, and clustering
mat, cluster_labels, cluster_labels_temp, valid_indicies, ds = open_and_process(data_path, k, tol, max_iter, init, n_init, var_name, tau_var_name, ht_var_name, lat_var_name, lon_var_name, height_or_pressure, wasserstein_or_euclidean, premade_cloud_regimes, lat_range, lon_range, time_range, only_ocean_or_land, land_frac_var_name, cluster=True, gpu=gpu)
# Plotting histograms
plot_hists(cluster_labels, k, ds, ht_var_name, tau_var_name, valid_indicies, mat, cluster_labels_temp, height_or_pressure, save_path)
# Plotting RFO
plot_rfo(cluster_labels, k ,ds, save_path)

## Further Analysis
This is all well and good, but this analysis only goes so far. It's likely a user will want to do further analysis beyond looking at the spatial distributions of the CRs. In this section you will learn how to do this. 

Described above, we create an array called cluster_labels that holds the cluster that each histogram belongs to. Here's how to use it:

First lets re-open the dataset specified by the data_path variable, since all the variables other than the one stored in var_name are automatically removed. 

In [ ]:
ds = xr.open_mfdataset(data_path)
ds

The optical depth - cloud top height joint histograms should be stored in ds under the variable entered as var_name, such that ds[var_name] opens the raw histogram data. For ISCCP, this data is in the shape (n_time, n_tau, n_pressures, n_lat, n_lon) where n_time, n_tau, n_pressures... are the number of times, tau bins, pressure bins etc that are present in the dataarray. Other data products will be roughly similar to this but may have the dimensions in a different order. cluster_labels is an array of integers in (almost) this same shape, but without n_tau and n_pressure dimensions. So, in this case cluster labels is shape (n_time, n_lat, n_lon). The value of cluster_labels[specific_time, specific_lat, specific_lon] will be an integer ranging from 0 to k-1 that indicates which CR the histogram located at ds[var_name][specific_time,:,:, specific_lat, specific_lon] belongs to. This may be slightly confusing, but a value of 0 indicates this histogram belongs to CR1, a value of 1 indicates it belongs to CR2, 2 to CR3 etc... This indexing works better for some operations in python, but if it truly displeases you, there is a very easy fix. After all the pre-written function calls but before starting your analysis, simply add the line "cluster_labels += 1". This will reindex the values to run from 1 through k, where each value is directly equal to the CR the corresponding histogram has been labeled as. It is also important to note that any histogram that contains a NaN will not be assigned a cluster, and its corresponding value in cluster labels will also be NaN.

Now that we understand the structure of the cluster_labels array, let’s go over how to use it. As can be seen above, our ds has two variables: n_pctaudist which holds the histogram data, and pc which holds the mean cloud-top pressure for cloudy pixels in each histogram. We will use the cluster labels array and the pc variable to calculate the mean cloud-top pressure for each CR. (Yes, this could also be calculated from the raw histogram data, but here we use a separate variable to illustrate how to do this for a variable that can’t simply be derived from the histogram data.)

Now, let’s create a dataarray of the mean cloud-top pressures.

In [ ]:
pc = ds.pc

Then we can loop through each CR and calculate the average cloud top pressure. We do this with the dataarray.where() function provided by xarray. This function (used on line 2 in the following cell) allows us to grab all the values of pc where cluster_labels is equal to a certain cluster and mask out all the other values. We can then take the mean of this and get the mean cloud top pressure for a certain CR. 

This method can be extended to be used on any variable, not just average cloud top pressure. It is also possible to remove the for loop in the below cell and preform this operation using only broadcasting and vectorization if one wishes. 

In [ ]:
# Looping through each cluster
for cr in range(k):
    # Grabbing the values of pc for histograms assigned to cr
    pc_of_cr = pc.where(cluster_labels == cr)
    # Calculating the mean cloud top pressure
    average_cloud_top_pressure = np.nanmean(pc_of_cr)
    # Printing results
    print(f"The average cloud top pressure of CR{cr+1} is {average_cloud_top_pressure} ")

## Summary
You've now learned to create and analyze cloud regimes! This process can be used on any data product that contains joint cloud top height and optical depth histograms, not just ISCCP. So go out there and get clustering on your own data!